# Seasonal Polar Plots of TOTALDEMAND - Month and Season

Uses nsw_df.parquet as base file for data

In [28]:
import plotly.express as px
import pandas as pd

In [94]:
link = './../data/NSW/nsw_df.parquet'
nsw_df = pd.read_parquet(link)
nsw_df.head(2)

,TOTALDEMAND,FORECASTDEMAND,TEMPERATURE,rrp,daily_avg_actual,daily_avg_forecast,forecast_error,smoothed_forecast_demand,year,quarter,month,dow,doy,hour,season,smoothed_total_demand,smoothed_temperature,minutes_past_midnight
2010-01-02 00:30:00,7343.30,7216.295694,24.3,20.67,NaN,NaN,91.620905,7699.974365,2010,1,1,5,2,0,1,7781.995417,25.233333,30
2010-01-02 01:00:00,7099.73,6977.373562,24.2,19.27,NaN,NaN,91.620905,7690.566078,2010,1,1,5,2,1,1,7773.996250,25.266667,60


In [98]:
df = nsw_df.groupby(pd.Grouper(freq='ME'), dropna=False).mean() #Monthly Aggregation
df['year'] = df.index.year.astype('category')
df['month'] = df.index.month.astype('category')
df['week'] = df.index.Week.astype('category')
df['day'] = df.index.day.astype('category')
df['hour'] = df.index.hour.astype('category')

AttributeError: 'DatetimeIndex' object has no attribute 'Week'

In [99]:
df['seasonName'] = df['month'].map(lambda m: 'Summer' if m in [1, 2, 12] else ('Autumn' if m in [3, 4, 5] else ('Winter' if m in [6, 7, 8] else 'Spring'))).astype('category')

df['monthName']= df.index.month_name().str[:3].astype('category')
df['dayName'] = df.index.day_name().str[:3].astype('category')
df['hour'] = df.index.hour.astype('category')


df['month'] = pd.to_datetime(df['month'], format='%m').dt.strftime('%b').astype('category')

df['year'] = df.index.year
df = df[df['year'] % 2 == 0] #Filter for even years only
df['year'] = df['year'].astype('category')

In [100]:
df.head(3)

,TOTALDEMAND,FORECASTDEMAND,TEMPERATURE,rrp,daily_avg_actual,daily_avg_forecast,forecast_error,smoothed_forecast_demand,year,quarter,month,dow,doy,hour,season,smoothed_total_demand,smoothed_temperature,minutes_past_midnight,season_name,month_name
2010-01-31,8993.321418,8978.120175,23.349803,51.437887,9025.952802,9017.270951,-12.397903,8971.854811,2010,1.0,Jan,3.165393,16.510076,11.507992,1.0,8981.270765,23.361962,705.489924,Summer,January
2010-02-28,9167.571458,9193.617117,23.335293,55.737232,9167.571458,9193.617117,36.414507,9185.923661,2010,1.0,Feb,3.000000,45.500000,11.500000,1.0,9173.056711,23.383526,705.000000,Summer,February
2010-03-31,8712.444422,8757.477408,21.616123,25.854603,8712.444422,8757.477408,46.495235,8752.905560,2010,1.0,Mar,2.806452,75.000000,11.500000,2.0,8706.214386,21.666356,705.000000,Autumn,March


In [89]:
def polar_plot(df=nsw_df, r='TEMPERATURE', t='month', c='year', title='Yearly'):
    fig = px.line_polar(df, r=r, theta=t, color=c, line_close=True, 
                        color_discrete_sequence=px.colors.sequential.Plasma_r,
                        template="plotly")
    fig.add_annotation(x=0.5, y=0.55, text='Temperature (°C)', showarrow=False, font=dict(size=10, color='black')) #Adds temperature label
    fig.update_layout(title=f'{title.title()} Seasonality of {r.title()}', title_x=0.5, polar=dict(radialaxis=dict(tickfont=dict(size=10))))
    
    fig.update_layout(legend=dict(x=0.9,y=0.9, traceorder='normal', font=dict(size=10))) #Move Legend
    fig.update_layout(width=800, height=600)
    
    return fig

In [128]:
df = nsw_df.groupby(pd.Grouper(freq='ME'), dropna=False).mean() #Monthly Aggregation
df['year'] = df.index.year.astype('category')
df['monthName']= df.index.month_name().str[:3].astype('category')
df = df.loc[(df.index.year % 2 == 0)] #2010-2020 only even years

polar_plot(df=df, r='TEMPERATURE', t='monthName', c='year', title='Yearly')

In [126]:
df = nsw_df.groupby(pd.Grouper(freq='h'), dropna=False).mean() #Hourly Aggregation
df['day'] = df.index.day.astype('category')
df['hourName']= df.index.hour.astype('string').astype('category')
df = df.loc['2012-03-01':'2012-03-06']

In [127]:
polar_plot(df=df, r='TEMPERATURE', t='hourName', c='day', title='Daily')

In [78]:
#TOTALDEMAND PolarPlots
if False: 
    fig = px.line_polar(df, r="TOTALDEMAND", theta="month", color="year", 
                        line_close=True,
                        color_discrete_sequence=px.colors.sequential.Plasma_r,
                        template="plotly_dark")
    fig.update_layout(polar=dict(radialaxis=dict(tickfont=dict(size=10)))) #Decreases radial axis font size
    fig.show()
    
    
    fig = px.line_polar(df, r="TOTALDEMAND", theta="season", color="year", 
                        line_close=True,
                        color_discrete_sequence=px.colors.sequential.Plasma_r,
                        template="plotly_dark")
    fig.update_layout(polar=dict(radialaxis=dict(tickfont=dict(size=10)))) #Decreases radial axis font size
    fig.show()  